In [48]:
import math

https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/

This notebook will walk through this article on yield and generators

A Python *generator* is a function which returns a *generator iterator* by calling **yield**. The next time `next()` is called on the generator iterator, the generator resumes execution *from where it called `yield`, not from the beginning of the function*. 

Most functions return a single value, but sometimes it's better to have a function that yields a series of values instead. This requires the function to "save its work", in a way, rather than restarting from the top every time you call it. 

In [2]:
for x in range(20):
    print(x)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


Every time I call the above function, it starts from the top and prints out all 20 values. The function can't save its place at say, 10, and then move on. 

A function that *could* do this wouldn't really "return" anything in the normal sense. `return` implies that the function is returning control of execution to the point where the function was called. In contrast, "yield" implies that the *transfer of control is temporary and voluntary*, and our function expects to regain it in the future.

These kinds of functions are known as `generators`. Outside of Python, these functions are generally referred to instead as `coroutines`, but all coroutines in Python is still a generator.

As an example, trying to write a function for prime numbers. If we ran a function on a very large list of numbers, the list would be so large that all the system's memory would be taken up. But the chokepoint is that a normal function **only gets one change to return results, and thus must return all results at once**. A `get_primes` function might be able to better handle lists of large numbers if it could just return the *next* value instead of all the values at once. If it was able to do this, it wouldn't need to create a list at all. 

If we tried doing something like this with a normal function, it'd get stuck, because the conditionality would cut it off early on and then it would just stick there. Instead of `return`, we need a way to generate a value, and when asked for the next one, pick up where we left off. Normal functions cannot do this, because when they return, they're done for good. Can't start it up again from a specific line. **Functions have a single entry point: the first line**. 

## Enter Generators

A generator function is defined like a normal function, but whenever it needs to generate a value, it does so with the **yield** keyword rather than **return**. If the body of a `def` contains the `yield` function, the function automatically becomes a `generator function`. 

In [10]:
def simple_generator_function():
    for i in range(10):
        yield(i)

In [11]:
for value in simple_generator_function():
    print(value)

0
1
2
3
4
5
6
7
8
9


In [12]:
our_generator = simple_generator_function()

In [21]:
next(our_generator)

8

Every time a `generator function` calls `yield`, the state of the generator is frozen: the values of all variables are saved and the next line of code to be executed is recorded until `next()` is called again. Once it is, the generator function picks up where it left off. If `next()` is never called again, the state recorded during the `yield` call is (eventually) discarded.

In [22]:
def get_primes(number):
    while True:
        if is_prime(number):
            yield number
        number += 1

If a `generator function` calls `return` or reaches the end of its definition, a `StopIteration` exception is raised, signalling that the generator is exhausted. That's why there's a `while True` loop is present in `get_primes`. If it weren't, the first time `next()` was called, we would check if the number is prime and possible yield it. If `next()` were called again, it would add 1 to number and hit the end of the generator function. 

So the `while` loop is there to make sure we *never* reach the end of the generator. The following will *not* work:

In [23]:
our_generator = simple_generator_function()

In [24]:
for value in our_generator:
    print(value)

0
1
2
3
4
5
6
7
8
9


In [25]:
print(next(our_generator))

StopIteration: 

Why doesn't it work? The generator was already exhausted when it was iterated through. But we can always create a new generator by calling the function again.

In [38]:
our_generator = simple_generator_function()

In [39]:
print(next(our_generator))

0


In [44]:
def is_prime(number):
    if number > 1:
        if number == 2:
            return True
        if number % 2 == 0:
            return False
        for current in range(3, int(math.sqrt(number) + 1), 2):
            if number % current == 0:
                return False
        return True
    return False

In [45]:
def get_primes(number):
    while True:
        if is_prime(number):
            yield number
        number += 1

In [46]:
def solve_number_10():
    total = 2
    for next_prime in get_primes(3):
        if next_prime < 2000000:
            total += next_prime
        else:
            print(total)
            return

1. Enter the `while` loop on line 3.
2. `if` condition holds (3 is prime). 
3. Yield the value 3 and control back to `solve_number_10`

Then, back in `solve_number_10':
1. The value 3 is passed back to the for loop.
2. The for loop assigns `next_prime` to this value.
3. `next_prime` is added to `total`. 
4. The for loop requests the next element from `get_primes`

Except this time, instead of entering `get_primes` back at the top and starting all over again, we resume at line 5, where we left off. `number` will *still have the same value it did when we called `yield`*. So, `number` is then incremented to 4, we hit the top of the `while` loop, and keep incrementing `number` until we hit the next prime number. This cycle will continue until the for loop stops at the first prime greater than 2,000,000.

In [49]:
solve_number_10()

142913828922


Holy cow, that was fast.

PEP 342 added support for passing values *into* generators. So generators now have the power to yield a value, *receive* a value, or both yield a value *and* receive a (possibly different) value in a single statement.

In [51]:
def get_primes(number):
    while True:
        if is_prime(number):
            number = yield number
        number += 1

`other = yield foo1` means, "yield `foo` and when a value is sent to me, set `other` to that value." You can send values to a generator by using the generators `send` method. 

In [53]:
def print_successive_primes(iterations, base=10):
    prime_generator = get_primes(base)
    prime_generator.send(None)
    for power in range(iterations):
        print(prime_generator.send(base ** power))

In [55]:
print_successive_primes(10)

2
11
101
1009
10007
100003
1000003
10000019
100000007
1000000007


We're printing the result of `generator.send`, which is possible because `send` both sends a value to the generator *and* returns the value yielded by the generator. So we're sending it a number equivalent to the base raised to some power, `get_primes` then loops through until it finds the next prime from that base number, and yields that number, which is returned by the same `send` function!

`prime_generator.send(None)` line. When using send to start a generatr, you must send `None`. The generator hasn't gotten to the first `yield` statement yet, so if we sent a real value, there would be nothing to receive it. The generator has to get started up first.